## Running SQL queries on Jupyter notebook

Note: https://ploomber.io/blog/sql-on-jupyter/

In [ ]:
%pip install jupysql --quiet
%pip install grpcio --quiet
%pip install grpcio-status --quiet

JupySQL allows you to run SQL in Jupyter/IPython via %sql and %%sql magics.

In [ ]:
%load_ext sql

In [ ]:
%%sql sqlite://
CREATE TABLE languages (name, rating, change);
INSERT INTO languages VALUES ('Python', 14.44, 2.48);
INSERT INTO languages VALUES ('C', 13.13, 1.50);
INSERT INTO languages VALUES ('Java', 11.59, 0.40);
INSERT INTO languages VALUES ('C++', 10.00, 1.98);

In [ ]:
%sql SELECT * FROM languages

In [ ]:
result = %sql SELECT * FROM languages WHERE rating > 10

In [ ]:
result

In this section, we’ll learn how to create a table and query the database using the to_sql and read_sql functions provided by Pandas. This will be demonstrated on Seaborn’s tips dataset. First, install the below packages:

In [ ]:
%pip install pandas seaborn memory-profiler --quiet

Now let’s use the to_sql function for saving the dataset in the database as a table.

In [1]:
import seaborn as sb

df = sb.load_dataset('tips')
conn = "sqlite:///example"
df.to_sql(
            'tips',            
            conn,                
            if_exists='replace'
           )

244

Let’s use the read_sql function to filter out some columns from this table.

In [2]:
import pandas as pd
query_result = pd.read_sql("""
            SELECT total_bill, tip, sex
            FROM tips
            WHERE time='Dinner'
            """, con=conn)
query_result.head()

,total_bill,tip,sex
0,16.99,1.01,Female
1,10.34,1.66,Male
2,21.01,3.50,Male
3,23.68,3.31,Male
4,24.59,3.61,Female


Running read_sql queries on large datasets can easily lead to out-of-memory issues. This function supports a chunksize argument, however, this argument is only used for creating chunks of dataframes to be returned to the user. By default, it will fetch all data into memory at once before chunking unless the database driver has a feature to support server-side-cursors.

Let’s add the above code to a Python script named read_tips.py and run a memory profiler to see the memory usage:

In [3]:
!python -m memory_profiler read_tips.py

Could not find script read_tips.py


Now set the chunksize parameter and check that the memory usage remains the same:

In [ ]:
!python -m memory_profiler read_tips_chunksize.py

## Database drivers
Database drivers like psycopg2 and sqlite3 can also be directly used for running SQL queries from notebooks. Let’s see a simple example using sqlite3:

First, we need to create a new database and call sqlite3.connect() to create a connection to the database tutorial.db in the current working directory.

In [ ]:
import sqlite3
con = sqlite3.connect("tutorial.db")

The con represents the connection to the on-disk database. We will also need to use a database cursor to execute SQL statements.

In [ ]:
cur = con.cursor()
cur.execute("CREATE TABLE movie(title, year, score)")
cur.execute("""
    INSERT INTO movie VALUES
        ('Monty Python and the Holy Grail', 1975, 8.2),
        ('And Now for Something Completely Different', 1971, 7.5)
""")
con.commit()
res = cur.execute("SELECT score FROM movie")
res.fetchall()